In [1]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 데이터 불러오기
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

# 데이터 불러오기
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
# 데이터 전처리 함수 정의
def preprocess_data(df):
    # 결측치 처리
    df['Age'] = df['Age'].fillna(df['Age'].mean())  # 평균으로 결측치 대체
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())  # 평균으로 결측치 대체
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])  # 최빈값으로 결측치 대체

    # 가족 크기 특성 생성
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1  # 가족 크기 계산

    # 범주형 변수 인코딩
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})  # 성별 인코딩
    df['Embarked'] = df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})  # 탑승 항구 인코딩

    # Pclass를 SES로 인코딩
    df['Pclass'] = df['Pclass'].map({1: 'Upper', 2: 'Middle', 3: 'Lower'})
    df['Pclass'] = pd.Categorical(df['Pclass']).codes  # 범주형 변수를 숫자로 변환

    # 사용할 특성 선택
    features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'FamilySize']
    return df[features]

In [5]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'FamilySize']

In [6]:
# 학습 데이터 전처리
X = preprocess_data(train)
y = train['Survived']

# 데이터 스케일링
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 학습 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# PyTorch 데이터셋 및 데이터로더 생성
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)

In [7]:
# 신경망 모델 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8, 32)  # 입력층 (7개 특성) -> 은닉층1 (32개 노드)
        self.fc2 = nn.Linear(32, 16) # 은닉층1 -> 은닉층2 (16개 노드)
        self.fc3 = nn.Linear(16, 1)  # 은닉층2 -> 출력층 (1개 노드, 생존 여부)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x


In [8]:
# 모델, 손실 함수, 옵티마이저 생성
model = Net()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train.unsqueeze(1))
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.6487
Epoch [20/100], Loss: 0.6320
Epoch [30/100], Loss: 0.6130
Epoch [40/100], Loss: 0.5900
Epoch [50/100], Loss: 0.5642
Epoch [60/100], Loss: 0.5379
Epoch [70/100], Loss: 0.5129
Epoch [80/100], Loss: 0.4908
Epoch [90/100], Loss: 0.4725
Epoch [100/100], Loss: 0.4583


In [9]:

# 검증 데이터로 모델 평가
with torch.no_grad():
    outputs = model(X_val)
    predicted = (outputs > 0.5).float()
    accuracy = (predicted == y_val.unsqueeze(1)).sum().item() / len(y_val)
    print(f'Validation Accuracy: {accuracy:.4f}')

# 테스트 데이터 전처리 및 예측
test_processed = preprocess_data(test)
test_processed = scaler.transform(test_processed)
test_processed = torch.tensor(test_processed, dtype=torch.float32)


Validation Accuracy: 0.8097


In [10]:
with torch.no_grad():
    test_outputs = model(test_processed)
    test_pred = (test_outputs > 0.5).int().numpy().squeeze()

# 제출 파일 생성
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': test_pred
})
submission.to_csv('submission_pytorch.csv', index=False)
print('\n제출 파일이 생성되었습니다.')


제출 파일이 생성되었습니다.
